In [ ]:
#pip install streamlit streamlit_folium openai python-dotenv geopandas


  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   --- ------------------------------------ 1.8/19.2 MB 14.4 MB/s eta 0:00:02
   --------- ------------------------------ 4.5/19.2 MB 12.2 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/19.2 MB 9.3 MB/s eta 0:00:02
   -------------- ------------------------- 7.1/19.2 MB 8.9 MB/s eta 0:00:02
   -------------------- ------------------- 9.7/19.2 MB 9.4 MB/s eta 0:00:02
   ------------------------ --------------- 11.8/19.2 MB 9.6 MB/s eta 0:00:01
   ----------------------------- ---------- 14.2/19.2 MB 9.8 MB/s eta 0:00:01
   -------------------------------- ------- 15.5/19.2 MB 9.5 MB/s eta 0:00:01
   --------------------------------- ------ 16.3/19.2 MB 8.7 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/19.2 MB 8.5 MB/s eta 0:00:01
   -----------------------------

In [1]:
import json
import sys
import os

In [2]:
# Esta forma es más robusta asumiendo que el notebook está en src_chatbot:
current_notebook_directory = os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd())) #Define la ruta actual
project_root = os.path.abspath(os.path.join(current_notebook_directory, '..')) #Sube un nivel (src_chatbot)

if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Raíz del proyecto añadida a sys.path: {project_root}")
print(f"Contenido de sys.path (primeros elementos): {sys.path[:5]}")


# Ahora puedes importar usando la ruta desde la raíz del proyecto
try:
    from chatbot_logic import (
        load_chunks_from_jsonl,
        normalize_text,
        extract_query_parameters,
        find_relevant_chunks,
        system_prompt_v2  # Si necesitas el prompt para algo
        # send_question_to_openai # Descomenta si quieres probarla y tienes un cliente OpenAI
    )
    print("Importación exitosa desde src_chatbot.chatbot_logic")
except ImportError as e:
    print(f"Error en la importación: {e}")
    print("Asegúrate de que __init__.py existe en la carpeta src_chatbot (ya lo tienes).")
    print("Y que la estructura de carpetas es la correcta.")

Raíz del proyecto añadida a sys.path: c:\Users\LENOVO\Documents\GitHub\qlab_chatbot_corrupcion
Contenido de sys.path (primeros elementos): ['c:\\Users\\LENOVO\\anaconda3\\envs\\zoom\\python311.zip', 'c:\\Users\\LENOVO\\anaconda3\\envs\\zoom\\DLLs', 'c:\\Users\\LENOVO\\anaconda3\\envs\\zoom\\Lib', 'c:\\Users\\LENOVO\\anaconda3\\envs\\zoom', '']
Importación exitosa desde src_chatbot.chatbot_logic


In [3]:
# Cargar los chunks una vez
all_docs_chunks = load_chunks_from_jsonl() # Asegúrate que la ruta sea correcta
print(f"Cargados {len(all_docs_chunks)} chunks.")
if all_docs_chunks:
    print("Ejemplo del primer chunk:")
    print(json.dumps(all_docs_chunks[0], indent=2, ensure_ascii=False))
else:
    print("¡No se cargaron chunks! Verifica la ruta y el archivo.")


Cargados 10620 chunks desde c:\Users\LENOVO\Documents\GitHub\qlab_chatbot_corrupcion\output\salida_chunks_final.jsonl
Cargados 10620 chunks.
Ejemplo del primer chunk:
{
  "chunk_id": "001_2016_2_0335_obj_0",
  "report_id": "001_2016_2_0335",
  "chunk_text": "Áncash\" se ejecuto de acuerdo a normatividad vigente.",
  "source_field": "objetivo",
  "metadata": {
    "numero_informe": "001-2016-2-0335",
    "titulo_informe": "Adquisición  de  Geomembrana  HDPE  Obra:  \"Instalación  de  la  línea  emisora  reubicación  de  la  planta  de  tratamiento  de  aguas  servidas  de  la  ciudad  Casma  -  Provincia de Casma - Áncash\"",
    "region": "ANCASH",
    "provincia": "CASMA",
    "distrito": "CASMA",
    "entidad_auditada": "MUNICIPALIDAD PROVINCIAL DE CASMA",
    "year": 2016,
    "periodo_inicio": null,
    "periodo_fin": null,
    "monto_auditado": 2133933.42,
    "monto_examinado": 2133933.42,
    "modalidad": "AC",
    "fecha_emision": "2016-08-31",
    "unidad_emite": "OCI - MUNICI

In [4]:
test_text = "Región Áncash con Énfasis"
normalized = normalize_text(test_text)
print(f"Original: '{test_text}' -> Normalizado: '{normalized}'")
# Esperado: region ancash con enfasis


Original: 'Región Áncash con Énfasis' -> Normalizado: 'region ancash con enfasis'


In [4]:
print("\n--- Probando extract_query_parameters ---")
queries = [
    "informe de corrupcion en lima 2019",
    "tienes datos sobre la region cuzco",
    "junin en 2020",
    "reportes de ayacucho y apurimac entre 2018 y 2020 sobre obras publicas",
    "que paso en la municipalidad de huamanga" # Ejemplo sin año/región clara
]

for q in queries:
    params = extract_query_parameters(q)
    print(f"Query: '{q}'\n  Params: {params}")



--- Probando extract_query_parameters ---
Query: 'informe de corrupcion en lima 2019'
  Params: {'years': ['2019'], 'regions': ['lima'], 'keywords': [], 'is_specific_enough': True}
Query: 'tienes datos sobre la region cuzco'
  Params: {'years': [], 'regions': ['cusco'], 'keywords': ['tienes', 'datos'], 'is_specific_enough': True}
Query: 'junin en 2020'
  Params: {'years': ['2020'], 'regions': ['junin'], 'keywords': [], 'is_specific_enough': True}
Query: 'reportes de ayacucho y apurimac entre 2018 y 2020 sobre obras publicas'
  Params: {'years': ['2020', '2018'], 'regions': ['ayacucho', 'apurimac'], 'keywords': ['reportes', 'entre', 'obras', 'publicas'], 'is_specific_enough': True}
Query: 'que paso en la municipalidad de huamanga'
  Params: {'years': [], 'regions': [], 'keywords': ['paso', 'huamanga'], 'is_specific_enough': True}


In [6]:
# Celda 4: Probar find_relevant_chunks (con los prints de depuración activados dentro de la función)
print("\n--- Probando find_relevant_chunks ---")

# Pregunta 1
question1 = "informe de corrupcion en lima 2019"
print(f"\nBuscando para: '{question1}'")
result1 = find_relevant_chunks(question1, all_docs_chunks, max_chunks=5)
if result1["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result1["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result1.get('no_data_for_filter') or result1.get('no_data_for_filter_after_score', 'desconocida')}")
    if result1.get("params"): print(f"  Parámetros usados: {result1.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result1['chunks'])} chunks.")
    # for i, chunk in enumerate(result1["chunks"]):
    #     print(f"  Chunk {i+1} (Informe: {chunk.get('metadata', {}).get('numero_informe', 'N/A')}, Región: {chunk.get('metadata', {}).get('region', 'N/A')}, Año: {chunk.get('metadata', {}).get('year', 'N/A')})")
    #     print(f"    Texto: {chunk.get('chunk_text', '')[:150]}...") # Imprime un fragmento

# Pregunta 2
question2 = "tienes datos sobre la region cuzco"
print(f"\nBuscando para: '{question2}'")
result2 = find_relevant_chunks(question2, all_docs_chunks, max_chunks=5)
# ... (mismo manejo de resultados que para question1) ...
if result2["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result2["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result2.get('no_data_for_filter') or result2.get('no_data_for_filter_after_score', 'desconocida')}")
    if result2.get("params"): print(f"  Parámetros usados: {result2.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result2['chunks'])} chunks.")


# Pregunta 3 (para Junín)
question3 = "informes para junin en 2021"
print(f"\nBuscando para: '{question3}'")
result3 = find_relevant_chunks(question3, all_docs_chunks, max_chunks=5)
# ... (mismo manejo de resultados) ...
if result3["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result3["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result3.get('no_data_for_filter') or result3.get('no_data_for_filter_after_score', 'desconocida')}")
    if result3.get("params"): print(f"  Parámetros usados: {result3.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result3['chunks'])} chunks.")




--- Probando find_relevant_chunks ---

Buscando para: 'informe de corrupcion en lima 2019'
Resultado: Encontrados 5 chunks.

Buscando para: 'tienes datos sobre la region cuzco'
Resultado: Encontrados 5 chunks.

Buscando para: 'informes para junin en 2021'
Resultado: Encontrados 5 chunks.
